In [2]:
'''Methods Comparison: Heston FFT, Heston MC QE, Heston NN'''

import pandas as pd
import sys, os, pathlib
import joblib 
import time 

# directory corrente del notebook
NOTEBOOK_DIR = pathlib.Path(os.getcwd())  

# root del progetto = la cartella che contiene "src/"
ROOT = NOTEBOOK_DIR.parents[0]  

sys.path.insert(0, str(ROOT))

from src.Heston_FFT import heston_price_fft
from src.Heston_MC import heston_mc_qe_price
heston_nn = joblib.load("../model_ann_1.pkl")

c:\Users\giovanni.piva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.0 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\giovanni.piva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator MLPRegressor from version 1.7.0 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\giovanni.piva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Tryi

In [3]:
# HESTON PARAMS 
S0   = 105.0
K    = 115.0
m    = K / S0
T    = 0.14
r    = 0.065
q    = 0.0
v0     = 0.03
kappa  = 3.44
theta  = 0.082
sigma_v= 0.257
rho    = -0.44


# FFT PARAMS
alpha  = 1.5
fft_N  = 4096     
eta    = 0.1      

# MC PARAMS
mc_steps = 200
mc_paths = 150_000
seed     = 42

# NN DATASET
X_new = pd.DataFrame([{
    "m": m,
    "T": T,
    "r": r,
    "v0": v0,
    "kappa": kappa,
    "theta": theta,
    "sigma_v": sigma_v,
    "rho": rho
}])

_ = heston_nn.predict(X_new) # warm-up 

In [5]:
# FFT
t0 = time.perf_counter()
price_fft = heston_price_fft(S0, K, T, r, v0, kappa, theta, sigma_v, rho,
                             option_type="call", q=q, alpha=alpha, N=fft_N, eta=eta)
t1 = time.perf_counter()
dt_fft = t1 - t0
print(f"FFT:  {price_fft:.6f}  ({dt_fft*1e3:.2f} ms)")


# MC
t0 = time.perf_counter()
price_mc, stderr_mc = heston_mc_qe_price(S0, K, T, r, q, v0, kappa, theta, sigma_v, rho,
                                         steps=mc_steps, paths=mc_paths,
                                         antithetic=True, seed=seed, option_type="call")
t1 = time.perf_counter()
dt_mc = t1 - t0
print(f"MC  :  {price_mc:.6f}  ± {1.96*stderr_mc:.6f} (95% CI)  ({dt_mc:.2f} s)")


# NN
t0 = time.perf_counter()
price_nn = float(heston_nn.predict(X_new)[0])
t1 = time.perf_counter()
dt_nn = t1 - t0

print(f"NN  :  {price_nn:.6f}  ({dt_nn*1e3:.3f} ms)")

FFT:  0.502312  (5.83 ms)
MC  :  0.496737  ± 0.008726 (95% CI)  (3.77 s)
NN  :  0.609025  (1.090 ms)
